In [1]:
from qtuum.api_wrappers import QuantinuumAPI as QAPI
import time

machine = 'H1-2E'

qapi = QAPI(machine=machine)
print('Machine status:', qapi.machine, 'is', qapi.status())


Machine status: H1-2E is reserved


In [2]:
from pytket.extensions.quantinuum import QuantinuumBackend

machine = 'H1-2E'
backend = QuantinuumBackend(device_name=machine)
backend.login()

Enter your Quantinuum email:  jha3@llnl.gov
Enter your Quantinuum password:  ········


In [3]:
# Add Grover's iterations
def addGSIteration(qc, omega):
    # Add Oracle
    if omega==0:
        qc.S(0)
        qc.S(1)
        qc.CZ(0,1)
        qc.S(0)
        qc.S(1)
    elif omega==1:
        qc.S(0)
        qc.CZ(0,1)
        qc.S(0)        
    elif omega==2:
        qc.S(1)
        qc.CZ(0,1)
        qc.S(1)
    elif omega==3:
        qc.CZ(0,1)
    else:
        print("Wrong Oracle state provided!")
        return
    # Add Grover's diffusion
    qc.add_barrier([0, 1]) # add a barrier on all qubits and bits
    qc.H(0)
    qc.H(1)
    qc.Z(0)
    qc.Z(1)
    qc.CZ(0,1)
    qc.H(0)
    qc.H(1)
    qc.add_barrier([0, 1])

In [4]:
from pytket.circuit import Circuit, fresh_symbol
from pytket.circuit.display import render_circuit_jupyter

# The superposition gate
circuit = Circuit(2, name="4L GS")
circuit.H(0)
circuit.H(1)
circuit.add_barrier([0, 1])
render_circuit_jupyter(circuit)

In [5]:
omega = 2
n_it = 16
for i in range(n_it):
    addGSIteration(circuit, omega)
circuit.measure_all()
render_circuit_jupyter(circuit)

In [6]:
compiled_circuit = backend.get_compiled_circuit(circuit, optimisation_level=2)

In [7]:
n_shots = 128
backend.cost(compiled_circuit, n_shots=n_shots, syntax_checker='H1-1SC')

16.2128

In [8]:
handle = backend.process_circuit(compiled_circuit, n_shots=n_shots)
status = backend.circuit_status(handle)
print(status)

CircuitStatus(status=<StatusEnum.QUEUED: 'Circuit is queued.'>, message='{"name": "4L GS", "submit-date": "2023-07-21T22:14:28.263973", "result-date": null, "queue-position": null, "cost": null, "error": null}', error_detail=None, completed_time=None, queued_time=None, submitted_time=None, running_time=None, cancelled_time=None, error_time=None, queue_position=None)


In [9]:
result = backend.get_result(handle)

result

BackendResult(q_bits={},c_bits={c[1]: 0, c[0]: 1},counts=None,shots=[[ 64]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [  0]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [192]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [  0]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [  0]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [ 64]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [  0]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [  0]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]],state=None,unitary=None,density_mat

In [10]:
result = backend.get_result(handle)
print(result.get_distribution())

{(0, 0): 0.0390625, (0, 1): 0.9375, (1, 0): 0.015625, (1, 1): 0.0078125}
